# NLP Demo 

## Set up global variables and random_state

In [ ]:
import numpy as np
import torch
from matplotlib import pyplot as plt
from datetime import datetime
from dataoob.util import set_random_state

device = torch.device("cpu")
random_state = set_random_state(10)
date = datetime.now().strftime("%m-%d_%H:%M")

## Set up data loader

#### Pick Noise rate and data set

In [ ]:
import dataoob.dataloader.datasets.nlpsets  # Must import as NLP data sets aren't imported automatically
nlp_datasets = ["bbc", "imdb"]
dataset_name = nlp_datasets[0]

In [ ]:
from dataoob.dataloader import DataFetcher

# Equivalent arguments
fetcher = (
    DataFetcher(dataset_name, False, random_state)
    .split_dataset(100, 50, 50)  # No noise functions for NLP yet
)
num_points = len(fetcher.x_train)
label_dim = (1,) if fetcher.y_train.ndim == 1 else fetcher.y_train[0].shape

## Set up models

In [ ]:
# Import examples of appropriate Models
# TODO think of more Regression models
from dataoob.model.bert import BertClassifier

In [ ]:
catalog = {
    # Only one NLP model so far
    'bert': BertClassifier("distilbert-base-uncased", *label_dim).to(device),
}

#### Choose a model from the catalog

In [ ]:
model_name = "bert"
metric_name = "accuracy"
train_kwargs = train_kwargs = {"epochs": 50, "batch_size": 20} if model_name in ("ann", "logreg") else {}

pred_model = catalog[model_name]

#### Base line performance

In [ ]:
from dataoob.evaluator.api import metrics_dict
model = pred_model.clone()
x_train, y_train, x_valid, y_valid, *_ = fetcher.datapoints
model.fit(x_train, y_train, **train_kwargs)
metric = metrics_dict[metric_name]

metric(y_valid, model.predict(x_valid))

## Data Evaluators

#### Lots of imoprts for the many Data Evaluators

In [ ]:
from dataoob.dataval.ame import AME
from dataoob.dataval.dvrl import DVRL
from dataoob.dataval.influence import InfluenceFunctionEval
from dataoob.dataval.knnshap import KNNShapley
from dataoob.dataval.oob import DataOob
from dataoob.dataval.shap import LeaveOneOut
from dataoob.dataval.shap import BetaShapley, DataShapley
from dataoob.dataval.shap.banzhaf import DataBanzhaf

#### Set up a series of data evaluators

In [ ]:
data_evaluators = [
    AME(num_models=1500, random_state=random_state),
    DataOob(random_state=random_state),  # 1000 samples
    # DVRL(rl_epochs=3000, random_state=random_state, device=device),  # DVRL requires tensor inputs
    InfluenceFunctionEval(5000, random_state=random_state),
    DataBanzhaf(5000, random_state=random_state),
    BetaShapley(gr_threshold=1.05, min_samples=500, cache_name="cached", random_state=random_state),
    DataShapley(gr_threshold=1.05, min_samples=500, cache_name="cached", random_state=random_state),
]

In [ ]:
from dataoob.evaluator import ExperimentMediator, presets
exper_med = ExperimentMediator(
    fetcher=fetcher,
    data_evaluators=presets.dummy_evaluators,
    pred_model=pred_model,
    train_kwargs=train_kwargs,
    metric_name=metric_name
)

## Running experiments on the data values

In [ ]:
from dataoob.evaluator.exper_methods import remove_high_low, point_removal

# Saving the results
import os
output_dir = f"tmp/{dataset_name}/{date}/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

### Removing high values and low values

In [ ]:
fig = plt.figure(figsize=(15, 15))
df_resp, fig = exper_med.plot(remove_high_low, include_train=True, col=2)
df_resp.to_csv(f"{output_dir}/remove_high_low.csv")

#### Remove descending values

In [ ]:
fig = plt.figure(figsize=(15, 15))
df_resp, fig = exper_med.plot(point_removal, include_train=True, col=2, percentile=.05, order="descending")
df_resp.to_csv(f"{output_dir}/descending_remove.csv")

#### Remove ascending values

In [ ]:
fig = plt.figure(figsize=(15, 15))
df_resp, fig = exper_med.plot(point_removal, include_train=True, col=2, order="ascending")
df_resp.to_csv(f"{output_dir}/ascending_remove.csv")